# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def yellowgrayscale(img):
    #enhance yellow then find grayscale
    # Based on an improved version of the grayscale 
    # function contributed by sjamthe in github
    # https://github.com/sjamthe/Self-driving-Car-ND-Lane-Finding/blob/master/P1-submit.ipynb
           
    #RGB limits
    lower = np.array([80,80,40])
    upper = np.array([255,255,80])
    
    # Threshold the HSV image to get only yellow colors
    mask = cv2.inRange(img, lower, upper)
    #show_image('mask',mask)
 
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(img,img, mask= mask)
    res = cv2.addWeighted(res, 1.0, img, 1.0, 0)  
    res = grayscale(res)
 
    return res    
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    if len(img.shape) == 2:
        img = np.dstack((img, img, img))
        
#     print(lines)
    for line in lines:
        for x1,y1,x2,y2 in line:
            #print(x1, y1, x2, y2)
            if x1 >= 0 and x1 < img.shape[1] and \
                y1 >= 0 and y1 < img.shape[0] and \
                x2 >= 0 and x2 < img.shape[1] and \
                y2 >= 0 and y2 < img.shape[0]:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

run your solution on all test_images and make copies into the test_images directory).

In [3]:
# image = mpimg.imread('test_images/solidWhiteCurve.jpg')
# ysize = image.shape[0]
# xsize = image.shape[1]
# print('This image is:', type(image), 'with dimesions:', image.shape)
# input_image = np.copy(image)

# y_gray=yellowgrayscale(input_image)
# plt.imshow(y_gray)

In [4]:
# blurry_img=gaussian_blur(y_gray,5)
# plt.imshow(blurry_img)

In [5]:
# cannied_img = canny(blurry_img,50,150)
# plt.imshow(cannied_img)

In [6]:
# vertice_set = np.array([[(100,ysize),(475, 325), (525, 325), (xsize, ysize)]], dtype=np.int32)
# masked_img = region_of_interest(cannied_img, vertice_set )   
# plt.imshow(masked_img)   

In [7]:
# #def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap)
# houghed_img = hough_lines(masked_img, 2, np.pi/180., 45, 40, 100)
# plt.imshow(houghed_img)


# Test on Images
Now you should build your pipeline to work on the images in the directory "test_images"
You should make sure your pipeline works well on these images before you try the videos.

In [8]:
# PBG Pipeline

def get_lines (image):
    rho = 2
    theta = np.pi/180.
    threshold = 60
    min_line_len = 120
    max_line_gap = 100
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len,
                            maxLineGap=max_line_gap)    
    #print(lines)
    return lines

def linefit (line_points):
    # Compute y = mx + b for each two points
    #x1, y1, x2, y2 = line_points
    x1 = line_points[0]
    y1 = line_points[1]
    x2 = line_points[2]
    y2 = line_points[3]
#     print(x1, y1, x2, y2)
    
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
#     print(m)
#     print(b)
    return (m, b)

def get_lanemarks(img_shape, lines):
#     print(img_shape)
#     print(lines)
    min_slope=0.5
    max_slope=2.0
    
    m1=np.array([])
    b1=np.array([])

    m2=np.array([])
    b2=np.array([])
    
    min_y=img_shape[0]
#     print (min_y)
    
    for line_points in lines:
        # fitting to line equation
        (m,b) = linefit(line_points)
#         print(m,b)
        # filtering the line based on the slope
        assert not np.isnan(m)        
        if abs(m) > min_slope and abs(m) < max_slope:
            min_y = min(min_y, line_points[1])
            min_y = min(min_y, line_points[3])
#             print(min_y)
            
            # distinguish left and right by sign of the slope
            if (m > 0):
                m1 = np.append(m1, m)
                b1 = np.append(b1, b)
#                 print(m, m1,b1)
            else:
                m2 = np.append(m2, m)
                b2 = np.append(b2, b)
#                 print(m2,b2)
                
    # find average of two main lines
    m1_mean = np.mean(m1)
    b1_mean = np.mean(b1)
#     print(m1)
#     print(b1)
    
    m2_mean = np.mean(m2)
    b2_mean = np.mean(b2)
#     print (m2)
#     print (b2)
    
    # Coordinates of the intersection
    x_intersec = (b2_mean - b1_mean) / (m1_mean - m2_mean)
    
    
    y_intersec = m1_mean * x_intersec + b1_mean
    
#     print (x_intersec, y_intersec)
    
    # highest point of the line
    y_end = max(y_intersec, min_y)
    
    # x coordinate of the start of the line on bottom frame
    
    y1 = img_shape[0] - 1
    x1 = (y1 - b1_mean) / m1_mean
    y2 = img_shape[0] - 1
    x2 = (img_shape[0] - b2_mean) / m2_mean
    
    x_end_1 = (y_end - b1_mean) / m1_mean
    x_end_2 = (y_end - b2_mean) / m2_mean
    
    return np.array([[[x1, y1, x_end_1, y_end]], [[x2, y2, x_end_2, y_end]]]).astype(int)


def overlay_markers(lanemarkers, image):
    # First a black image
    lines_image = np.copy(image)*0
    # then the lines
    draw_lines(lines_image, lanemarkers, color=[255,0,0], thickness = 10)
    # blended lines
    image_out = weighted_img(lines_image, image)
    return image_out

def pipeline (img_input):
    
    # Prepare the image 
    
    y_gray=yellowgrayscale(img_input)
#     plt.subplot(2,2,1)
#     plt.imshow(y_gray)
    
    # Blurr the image
    blurry_img=gaussian_blur(y_gray,5)
#     plt.subplot(2,2,2)
#     plt.imshow(blurry_img)
    
    # get the edges using canny filter
    cannied_img = canny(blurry_img,50,150)
#     print(cannied_img.shape)
#     plt.subplot(3,3,3)
#     plt.imshow(cannied_img)
    
    # mask the region of interest
    vertice_set = np.array([[(0,540),
                             (400, 350), 
                             (600, 350),
                             (cannied_img.shape[1], 
                              cannied_img.shape[0])]], 
                            dtype=np.int32)
#     print(vertice_set)
    masked_img = region_of_interest(cannied_img, vertice_set)   
#     plt.subplot(3,3,4)
#     plt.imshow(masked_img)
    
    # Hough the Image
#     houghed_img = hough_lines(masked_img, 2, np.pi/180., 60, 120, 100)
#     plt.imshow(houghed_img)   
    
    # get the lines from the hough transformation
    lane_lines = get_lines(masked_img)
#     print (lane_lines)
    img_lines = np.copy(img_input)
#     plt.imshow(img_lines)
#     draw_lines(img_lines, lane_lines)
    
    
    # extract lanes for both sides
    lanemarkers = get_lanemarks(masked_img.shape, lane_lines.squeeze())
#     plt.subplot(3,3,5)
#     plt.imshow(lanemarkers)
    
#     # set overlayed markers
    image_output = overlay_markers(lanemarkers, img_input)
# #     plt.subplot(3,3,6)
#     plt.imshow(image_output)
    
#     #return img_lines
#     return image_output

    return image_output



    

In [9]:
import os
test_images = os.listdir("test_images/")
if not os.path.isdir("results"):
    os.mkdir("results")
#print(test_images)
    
    
for i in range(0, len(test_images)):
    images = mpimg.imread(os.path.join("test_images", test_images[i]))
    img_out = pipeline(images)
    mpimg.imsave(os.path.join("results", test_images[i]), img_out)

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [10]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [11]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    return result

Let's try the one with the solid white lane on the right first ...

In [12]:
white_output = 'White.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video White.mp4
[MoviePy] Writing video White.mp4


100%|█████████▉| 221/222 [00:07<00:00, 30.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: White.mp4 

CPU times: user 5.4 s, sys: 230 ms, total: 5.63 s
Wall time: 8.11 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [14]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(pipeline)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:22<00:00, 30.42it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 17.1 s, sys: 660 ms, total: 17.7 s
Wall time: 23.1 s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

Pipeline: 
- Preprocessing of the image
- Gaussian Blurr
- Canny Filter
- Masking
- Hough Filtering
- Extraction of the lanes by:
- defining max and min slopes
- separating left and right
- averaging the slopes of the sample lines at each side of the road

I perceived significant performance improvements through:
- usage of yellow colors on the preprocessing of the image
- reducing the height of the trapezoidal mask.



## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [16]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(pipeline)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:16<00:00, 15.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 10.4 s, sys: 540 ms, total: 11 s
Wall time: 18.2 s


In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))